In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from itertools import combinations
import matplotlib.pyplot as plt # importing matplotlib
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import statistics

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(3)]
subfolder_with_path[0] = main_directory + '/'+'ABAcluster_cropped' # cluster files with samples in columns to calulate volumes
subfolder_with_path[1] = main_directory + '/'+'ABAconsensus_cropped' # activity files
subfolder_with_path[2] = main_directory + '/'+'zzz_input_and_reference_files'

In [8]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"Sample": int,
                                      "Side": "string",
                                      "Marker": "string",
                                      "Condition": "string",
                                      "Ort_code":"string",
                                      "488min":"string",
                                      "xy_res":float,
                                      "z_res":float,
                                      "olf_bulb":float,
                                      "Exp_Dir":"string"})

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [10]:
#Saving drugs and markers' names from the "sample_overview" file

drugs = samples_overview['Condition'].unique()
markers = samples_overview['Marker'].unique()

d = 0
m = 0
counters_names = []
while (m < len(markers)):
    while (d < len(drugs)):
        counters_names.append(drugs[d] + '_' + markers[m])
        d += 1
    m += 1
    d = 0

In [14]:
# Changing directory to the "ABAcluster_cropped" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "ABAcluster_cropped" folder to obtain number of clusters 
# and counting the number of columns in each file to count the number of samples
# The clusters vector contains all clusters names
#The samples_and_clusters matrix contains all samples names ordered in agreement witht the clusters vector

clusters = os.listdir(subfolder_with_path[0]) 
number_of_clusters = len(clusters)
samples_and_clusters_matrix = ["" for x in range(number_of_clusters)]

i = 0
while i < number_of_clusters:
    samples = pd.read_csv(clusters[i],low_memory=False).columns.tolist()
    samples_and_clusters_matrix[i] = samples
    i += 1


In [15]:
samples_and_clusters_matrix

[['sample22',
  'sample24',
  'sample26',
  'sample28',
  'sample30',
  'sample32',
  'sample34',
  'sample36',
  'sample40',
  'sample44',
  'sample46',
  'sample48',
  'sample50',
  'sample52',
  'sample56',
  'sample58',
  'sample60',
  'sample62',
  'sample64',
  'sample66',
  'sample68',
  'sample72',
  'sample76',
  'sample78',
  'sample80',
  'sample82',
  'sample84',
  'sample86',
  'sample88',
  'sample90',
  'sample92',
  'sample94',
  'sample96'],
 ['sample22',
  'sample24',
  'sample26',
  'sample28',
  'sample30',
  'sample32',
  'sample34',
  'sample36',
  'sample40',
  'sample44',
  'sample46',
  'sample48',
  'sample50',
  'sample52',
  'sample56',
  'sample58',
  'sample60',
  'sample62',
  'sample64',
  'sample66',
  'sample68',
  'sample72',
  'sample76',
  'sample78',
  'sample80',
  'sample82',
  'sample84',
  'sample86',
  'sample88',
  'sample90',
  'sample92',
  'sample94',
  'sample96']]

In [18]:
file = pd.read_csv("cluster_1_ABA_histogram.csv")
# Saving cluster's number
filename_segmented = clusters[0].split('_')
cluster_number = filename_segmented[1]
cluster_number
    # Reading file's data
    
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

'1'

In [ ]:
#Using a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_of_clusters:

    ##############################################
    # Processing each file
    ##############################################
    
    file = clusters[i]
    # Saving cluster's number
    filename_segmented = file.split('_')
    cluster_number = filename_segmented[1]

    # Reading file's data
    
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(7)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]
   

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)


In [ ]:
columns=['a','b','c','d','e']
string =['PIXEL_COUNT_']
df = pd.DataFrame('x', index=range(3), columns=['a','b','c','d','e'])
df = df.add_suffix('_PIXEL_COUNT')

df